Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [1]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-10-29 22:36:14,351: INFO: running C:\Users\VICTUS\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"9e497c01-5e63-4626-9eac-8a81b0fe9f3f" --shell=9007 --transport="tcp" --iopub=9009 --f=c:\Users\VICTUS\AppData\Roaming\jupyter\runtime\kernel-v2-3532o9KkTCqd4Jpu.json
2023-10-29 22:36:35,964: INFO: Saved 10000 articles
2023-10-29 22:36:51,136: INFO: Saved 20000 articles
2023-10-29 22:37:04,157: INFO: Saved 30000 articles
2023-10-29 22:37:15,367: INFO: Saved 40000 articles
2023-10-29 22:37:27,242: INFO: Saved 50000 articles
2023-10-29 22:37:38,549: INFO: Saved 60000 articles
2023-10-29 22:37:49,782: INFO: Saved 70000 articles
2023-10-29 22:38:01,673: INFO: Saved 80000 articles
2023-10-29 22:38:22,254: INFO: Saved 90000 articles
2023-10-29 22:38:35,689: INFO: Saved 100000 articles
2023-10-29 22:38:47,169: INFO: Saved 110000 articles
2023-10-29 22:38:57,626: I

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [2]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-10-29 22:46:36,237: INFO: running C:\Users\VICTUS\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py --ip=127.0.0.1 --stdin=9008 --control=9006 --hb=9005 --Session.signature_scheme="hmac-sha256" --Session.key=b"9e497c01-5e63-4626-9eac-8a81b0fe9f3f" --shell=9007 --transport="tcp" --iopub=9009 --f=c:\Users\VICTUS\AppData\Roaming\jupyter\runtime\kernel-v2-3532o9KkTCqd4Jpu.json
2023-10-29 22:46:36,238: INFO: collecting all words and their counts
2023-10-29 22:46:36,240: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-10-29 22:46:38,923: INFO: PROGRESS: at sentence #10000, processed 10738098 words, keeping 353874 word types
2023-10-29 22:46:40,570: INFO: PROGRESS: at sentence #20000, processed 17334952 words, keeping 490368 word types
2023-10-29 22:46:41,800: INFO: PROGRESS: at sentence #30000, processed 22148080 words, keeping 591133 word types
2023-10-29 22:46:42,955: INFO: PROGRESS: at sentence #40000, processed 26549341 words, keeping 

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [3]:
import gensim
# import gensim.models.keyedvectors as word2vec
# model = gensim.models.Word2Vec.load(namaFileModel)
# model = word2vec.KeyedVectors.load_word2vec_format(namaFileModel, binary=True)

namaFileModel = "w2vec_wiki_id300_0.txt"

sim = model.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta:{}".format(sim))
sim = model.wv.most_similar("Bandung")
print("10 kata terdekat dari Bandung:{}".format(sim))
 
sim = model.wv.similarity("Yogyakarta", "Surakarta")
print("Kedekatan Yogyakarta-Surakarta: {}".format(sim))
sim = model.wv.similarity("Yogyakarta", "Semarang")
print("Kedekatan Yogyakarta-Semarang: {}".format(sim))
 
sim = model.wv.most_similar_cosmul(positive=['minuman', 'rendang'], negative=['makanan'])
print("makanan-rendang, minuman-?: {}".format(sim))
sim = model.wv.most_similar_cosmul(positive=['mobil', 'honda'], negative=['motor'])
print("motor-honda, mobil-?: {}".format(sim))

10 kata terdekat dari Jakarta:[('Kemang', 0.6043619513511658), ('Surabaya', 0.6042571067810059), ('Warteg', 0.5971762537956238), ('Kemayoran', 0.5806102156639099), ('Cilandak', 0.5594650506973267), ('Epicentrum', 0.5456141233444214), ('Ciputat', 0.5436855554580688), ('Warkop', 0.541252851486206), ('Menteng', 0.539467990398407), ('Palmerah', 0.538223385810852)]
10 kata terdekat dari Bandung:[('Cimahi', 0.7077953219413757), ('Bogor', 0.6847033500671387), ('Sukabumi', 0.6361453533172607), ('Tasikmalaya', 0.6332675814628601), ('Purwakarta', 0.6231706142425537), ('Jatinangor', 0.6032922267913818), ('Garut', 0.5998300313949585), ('Cianjur', 0.5975313186645508), ('Baleendah', 0.5920150279998779), ('Surabaya', 0.5890358686447144)]
Kedekatan Yogyakarta-Surakarta: 0.7995200753211975
Kedekatan Yogyakarta-Semarang: 0.6227614879608154
makanan-rendang, minuman-?: [('Minuman', 0.8568644510401198), ('Fanta', 0.8489759898069952), ('martabak', 0.8477477331982342), ('limun', 0.8470192782312961), ('tuak',